In [1]:
# Code to mount google drive and change working directory
from google.colab import drive
drive.mount('/content/drive')
%ls
%cd drive/
%ls
%cd My\ Drive
%ls

Mounted at /content/drive
drive/  sample_data/
/content/drive
MyDrive/
/content/drive/My Drive
 ClassTest/                      data/                              'SVM classifier'/
 Class_Test_A_ZuhayrUmar.docx    google_news_headlines_by_date.csv   test_Sentiment_Data.csv
 Class_Test_B_ZuhayrUmar.ipynb  'NN classifier'/
'Colab Notebooks'/               sentiment_analysis_results.csv


In [4]:
### Google News Headline Scrapper

In [5]:
!pip install feedparser

import pandas as pd
from datetime import datetime, timedelta
import feedparser

# Define your start and end dates for filtering
start_date = datetime(2024, 3, 28)
end_date = datetime(2024, 4, 4)

rss_url = "https://news.google.com/rss/search?q=%22S%26P+500%22+OR+%22SPX%22+OR+%22$SPY%22&hl=en-US&gl=US&ceid=US:en"
feed = feedparser.parse(rss_url)

articles_by_date = {}

for entry in feed.entries:
    published_date = datetime(*entry.published_parsed[:6])

    # Filter articles by the specified start and end dates
    if start_date <= published_date <= end_date:
        date_str = published_date.strftime('%Y-%m-%d')
        headline = entry.title

        if date_str in articles_by_date:
            articles_by_date[date_str].append(headline)
        else:
            articles_by_date[date_str] = [headline]

data_for_df = []
for date, headlines in articles_by_date.items():
    row = {'date': date}
    for i, headline in enumerate(headlines):
        row[f'headline_{i+1}'] = headline
    data_for_df.append(row)

df_articles = pd.DataFrame(data_for_df)
df_articles = df_articles.fillna('')

print(df_articles.head())

output_file_name = 'google_news_headlines_by_date.csv'
df_articles.to_csv(output_file_name, index=False)

print(f"Filtered and saved articles to {output_file_name}.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=d09ed45cf6b7ec696c018d4b71d9753b895ce3a6004049f46487d49afceb63a2
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
         date                                         headline_1  \
0  2024-04-03  S&P 500 Gains and Losses Today: Cosmetics Stoc...   
1  2024-04-02  S&P 500 Gains and Losses Today: Health Care St...   
2  2024-04-01  S&P 500 Gains and Losses Today: Shares of Memo...   
3  2024-03-31  The Magnificent 7 are no longer the only stock...   
4  2024-03-29  S&P 500 Surges to Start the Year - The New Yor...   

                                          headline_2  \
0  S&P 500, Nasdaq close slightly higher after so...   
1  Should You Buy the 3 Highest-Paying Dividend S...   
2  Micron I

In [6]:
### Sentiment Analysis Code

In [7]:
import pandas as pd

!pip install flair

import flair
import scipy.linalg
print(scipy.linalg.triu)
from textblob import TextBlob
import os
import datetime
import numpy as np

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

print("Sentiment analysis imports.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [11]:
import pandas as pd
import flair

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

negative_keywords = ['plunges', 'falls', 'down', 'losses', 'worst', 'declines', 'drops', 'suffers']

def simple_sentiment_heuristic(headline):
    """
    Simple heuristic to detect negative sentiment based on specific keywords.
    """
    for keyword in negative_keywords:
        if keyword in headline.lower():
            return "NEGATIVE", 1.0
    return None, None

def get_sentiment_label(sentence):
    """
    Determine sentiment label based on the Flair model prediction.
    """
    flair_sentiment.predict(sentence)
    sentiment_score = sentence.labels[0].score
    if sentence.labels[0].value == 'NEGATIVE':
        sentiment_score = 1 - sentiment_score
    if sentiment_score > 0.55:
        return "POSITIVE", sentiment_score
    elif sentiment_score < 0.45:
        return "NEGATIVE", sentiment_score
    else:
        return "NEUTRAL", sentiment_score

def get_sentiment_report(input_filename, output_filename):
    data_df = pd.read_csv(input_filename)
    results = []

    for index, row in data_df.iterrows():
        date = row['date']
        daily_sentiments = []
        sentiment_confidences = []

        for headline in row[1:].dropna():
            heuristic_label, heuristic_confidence = simple_sentiment_heuristic(headline)
            if heuristic_label:
                sentiment_label = heuristic_label
                sentiment_confidence = heuristic_confidence
            else:
                sentence = flair.data.Sentence(headline)
                sentiment_label, sentiment_confidence = get_sentiment_label(sentence)

            daily_sentiments.append(sentiment_label)
            sentiment_confidences.append(sentiment_confidence)

        overall_sentiment = "NEUTRAL"
        if daily_sentiments:
            positive_count = daily_sentiments.count("POSITIVE")
            negative_count = daily_sentiments.count("NEGATIVE")
            if positive_count > negative_count:
                overall_sentiment = "POSITIVE"
            elif negative_count > positive_count:
                overall_sentiment = "NEGATIVE"

        overall_confidence = sum(sentiment_confidences) / len(sentiment_confidences) if sentiment_confidences else 0

        results.append([date, overall_sentiment, overall_confidence] + daily_sentiments)

    max_headlines = max(len(r) - 3 for r in results)
    column_names = ['date', 'overall_sentiment', 'confidence'] + [f'headline_{i}_sentiment' for i in range(1, max_headlines + 1)]
    results_df = pd.DataFrame(results, columns=column_names)

    results_df.to_csv(output_filename, index=False)
    print(f"Sentiment analysis results saved to {output_filename}")

if __name__ == "__main__":
    input_filename = 'test_Sentiment_Data.csv'
    output_filename = 'sentiment_analysis_results_2.csv'
    get_sentiment_report(input_filename, output_filename)


2024-04-04 20:23:30,512 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpvrpo8e6o


100%|██████████| 253M/253M [00:15<00:00, 17.7MB/s]

2024-04-04 20:23:45,993 copying /tmp/tmpvrpo8e6o to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2024-04-04 20:23:46,636 removing temp file /tmp/tmpvrpo8e6o


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Sentiment analysis results saved to sentiment_analysis_results_2.csv
